In [1]:
%load_ext autoreload
%autoreload 2
%load_ext snakeviz
%load_ext line_profiler

# turn off autoreload so that we can use the old model 
# when editing the current project

from coeditor.common import *
import os

os.chdir(proj_root())

In [2]:
import jedi, parso
from spot.static_analysis import PythonProject, UsageAnalysis
from coeditor.usage_analysis import JediUsageAnalysis

In [4]:
from coeditor.code_change import *
from coeditor.history import get_commit_history

commits = get_commit_history(proj_root(),  5)
for c in commits:
    print(c)

CommitInfo(hash='f690f7e8f6403025258301a18f90f7df248b3a16', parents=('e2ec640b34d01934767888e94a49d0a4f8b43bf5',), msg='Improve PythonModule construction speed.')
CommitInfo(hash='e2ec640b34d01934767888e94a49d0a4f8b43bf5', parents=('5d3758c0a5a2d009ecf55ca7eff4917ddf33be52',), msg='Add doc string removal in `normalize_code_by_ast`.')
CommitInfo(hash='5d3758c0a5a2d009ecf55ca7eff4917ddf33be52', parents=('0f9c7276c7faff1e66acf6b5085054129a44af5a',), msg='Add option to output logs to `.coeditor_logs`.')
CommitInfo(hash='0f9c7276c7faff1e66acf6b5085054129a44af5a', parents=('db578d77857af11288eba44ea91f07b101cfa38e',), msg='push minor changes.')
CommitInfo(hash='db578d77857af11288eba44ea91f07b101cfa38e', parents=('a86da5585332f5a1eaa444d6e22324b1be651ad1',), msg='Save attention_mode in model config.')


In [ ]:
workdir = proj_root() / "../temp-1"
JProjectChange.edits_from_commit_history(proj_root(), commits, workdir)

In [6]:
%%timeit -n 1 -r 2
analysis = UsageAnalysis(pyproj, add_implicit_rel_imports=True, record_type_usages=True)

9.82 s ± 26 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [34]:
%%timeit -n 1 -r 2
PythonProject.from_root(proj_root())

4.79 s ± 21.6 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [17]:
jproj = jedi.Project(proj_root())
jmodules = dict()
for f in proj_root().glob("src/**/*.py"):
    s = jedi.Script(path=f, project=jproj)
    jmodules[f] = s._module_node

In [15]:
ex_code = """
import foo
x += 1
@staticmethod
def f():
    return bar()
"""
mod = parso.parse(ex_code)
print(mod.dump())

Module([
    PythonNode('simple_stmt', [
        ImportName([
            Keyword('import', (2, 0), prefix='\n'),
            Name('foo', (2, 7), prefix=' '),
        ]),
        Newline('\n', (2, 10)),
    ]),
    PythonNode('simple_stmt', [
        ExprStmt([
            Name('x', (3, 0)),
            Operator('+=', (3, 2), prefix=' '),
            Number('1', (3, 5), prefix=' '),
        ]),
        Newline('\n', (3, 6)),
    ]),
    PythonNode('decorated', [
        Decorator([
            Operator('@', (4, 0)),
            Name('staticmethod', (4, 1)),
            Newline('\n', (4, 13)),
        ]),
        Function([
            Keyword('def', (5, 0)),
            Name('f', (5, 4), prefix=' '),
            PythonNode('parameters', [
                Operator('(', (5, 5)),
                Operator(')', (5, 6)),
            ]),
            Operator(':', (5, 7)),
            PythonNode('suite', [
                Newline('\n', (5, 8)),
                PythonNode('simple_stmt', [
     

In [3]:
jproj = jedi.Project(proj_root())
s = jedi.Script(path=proj_root() / "src/coeditor/usage_analysis.py")
s.goto(28, 9)

[<Name name='self', description='param self'>]

In [5]:
from spot.static_analysis import PythonModule

%lprun -T output/lprof.txt -f PythonModule.from_cst pyproj = PythonProject.from_root(proj_root())


*** Profile printout saved to text file 'output/lprof.txt'. 


Timer unit: 1e-09 s

Total time: 20.8927 s
File: /home/jiayi/Projects/SPOT/src/spot/static_analysis.py
Function: from_cst at line 495

Line #      Hits         Time  Per Hit   % Time  Line Contents
   495                                               @staticmethod
   496                                               def from_cst(module: cst.Module, name: str, drop_comments: bool) -> "PythonModule":
   497        60 4516638349.0 75277305.8     21.6          wrapper = MetadataWrapper(module)
   498        60 8465727458.0 141095457.6     40.5          src_map = dict(wrapper.resolve(PositionProvider))
   499        60     151722.0   2528.7      0.0          module = wrapper.module
   500        60      18831.0    313.9      0.0          removed_comments = []
   501        60      19426.0    323.8      0.0          if drop_comments:
   502        60    6497932.0 108298.9      0.0              remover = CommentRemover(src_map=src_map)
   503        60 5250321636.0 87505360.6     25.1        

In [7]:
analysis = JediUsageAnalysis(pyproj)

%timeit -r 2 -n 1 full_usages = [analysis.get_module_usages(m) for m in pyproj.modules]

Analyzing spot.experiments.utils: 100%|██████████| 652/652 [00:00<00:00, 1886.30it/s]
Analyzing spot.experiments: 0it [00:00, ?it/s]
Analyzing coeditor.history: 100%|██████████| 1737/1737 [00:00<00:00, 2448.64it/s]
Analyzing coeditor: 0it [00:00, ?it/s]
Analyzing data.code.dummy.dummy_1: 100%|██████████| 5/5 [00:00<00:00, 3422.80it/s]
Analyzing data.code.dummy: 0it [00:00, ?it/s]
Analyzing tests.spot.test_func_decoding: 100%|██████████| 163/163 [00:00<00:00, 3749.62it/s]
Analyzing tests.spot: 0it [00:00, ?it/s]
Analyzing tests.spot.test_model_creation: 100%|██████████| 74/74 [00:00<00:00, 266.52it/s]
Analyzing tests: 0it [00:00, ?it/s]
Analyzing tests.coeditor.test_edits: 100%|██████████| 336/336 [00:00<00:00, 2463.88it/s]
Analyzing tests.coeditor: 0it [00:00, ?it/s]
Analyzing scripts.coeditor.train_model: 100%|██████████| 256/256 [00:00<00:00, 1720.92it/s]
Analyzing output.TinyEval.repos.test.ActivityWatch.util: 100%|██████████| 418/418 [00:00<00:00, 1060.29it/s]
Analyzing output.Tiny

23.9 s ± 4.99 s per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [8]:
analysis = JediUsageAnalysis(pyproj)

%snakeviz -t  analysis.get_module_usages("coeditor.retrieval_model")

Analyzing coeditor.retrieval_model: 100%|██████████| 3852/3852 [00:05<00:00, 703.05it/s] 


 
*** Profile stats marshalled to file '/tmp/tmpjoxzvb5s'.
Opening SnakeViz in a new tab...
snakeviz web server started on 127.0.0.1:8080; enter Ctrl-C to exit
http://127.0.0.1:8080/snakeviz/%2Ftmp%2Ftmpjoxzvb5s


In [13]:
from coeditor.usage_analysis import fast_goto
from jedi.inference.names import AbstractTreeName

analysis = JediUsageAnalysis(pyproj)
%lprun -T output/lprof.txt -f AbstractTreeName.goto full_usages = analysis.get_module_usages("coeditor.retrieval_model")

Analyzing coeditor.retrieval_model: 100%|██████████| 3852/3852 [00:05<00:00, 756.05it/s] 


*** Profile printout saved to text file 'output/lprof.txt'. 


Timer unit: 1e-09 s

Total time: 3.81149 s
File: /home/jiayi/Projects/SPOT/.venv/lib/python3.11/site-packages/jedi/inference/names.py
Function: goto at line 143

Line #      Hits         Time  Per Hit   % Time  Line Contents
   143                                               def goto(self):
   144      3932    1337116.0    340.1      0.0          context = self.parent_context
   145      3932     972752.0    247.4      0.0          name = self.tree_name
   146      3932   18773324.0   4774.5      0.5          definition = name.get_definition(import_name_always=True)
   147      2759     545903.0    197.9      0.0          if definition is not None:
   148      1173     212765.0    181.4      0.0              type_ = definition.type
   149       746     156554.0    209.9      0.0              if type_ == 'expr_stmt':
   150                                                           # Only take the parent, because if it's more complicated than just
   151                                

In [22]:
analysis = JediUsageAnalysis(pyproj)
%timeit -r 2 -n 1 full_usages = analysis.get_module_usages("spot.static_analysis")

Analyzing spot.static_analysis: 100%|██████████| 5012/5012 [00:02<00:00, 2243.74it/s]


2.38 s ± 144 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [8]:
[len(analysis.get_module_usages("spot.static_analysis")) for _ in range(4)]

Analyzing spot.static_analysis: 100%|██████████| 5012/5012 [00:02<00:00, 2138.55it/s]


[4145, 4145, 4145, 4145]

In [16]:
environment_path = Path("~/anaconda3/envs/bare_3.10").expanduser()
analysis = JediUsageAnalysis(pyproj)
less_usages = analysis.get_all_usages()
len(less_usages["coeditor.usage_analysis"])

Analyzing spot.experiments.utils: 100%|██████████| 652/652 [00:00<00:00, 1090.96it/s]
Analyzing spot.experiments: 0it [00:00, ?it/s]
Analyzing coeditor.history: 100%|██████████| 1737/1737 [00:00<00:00, 2382.86it/s]
Analyzing coeditor: 0it [00:00, ?it/s]
Analyzing data.code.dummy.dummy_1: 100%|██████████| 5/5 [00:00<00:00, 3160.74it/s]
Analyzing data.code.dummy: 0it [00:00, ?it/s]
Analyzing tests.spot.test_func_decoding: 100%|██████████| 163/163 [00:00<00:00, 5608.60it/s]
Analyzing tests.spot: 0it [00:00, ?it/s]
Analyzing tests.spot.test_model_creation: 100%|██████████| 74/74 [00:00<00:00, 5600.98it/s]
Analyzing tests: 0it [00:00, ?it/s]
Analyzing tests.coeditor.test_edits: 100%|██████████| 336/336 [00:00<00:00, 2794.87it/s]
Analyzing tests.coeditor: 0it [00:00, ?it/s]
Analyzing scripts.coeditor.train_model: 100%|██████████| 256/256 [00:00<00:00, 2070.34it/s]
Analyzing output.TinyEval.repos.test.ActivityWatch.util: 100%|██████████| 418/418 [00:00<00:00, 2868.07it/s]
Analyzing output.Tin

total usages: 31207


219

In [19]:
def print_dict(xs: dict, path: Path):
    with path.open("w") as f:
        for k, v in xs.items():
            print(f"{k}: {v}", file=f)

print_dict(less_usages["coeditor.usage_analysis"], Path("less_usages.txt"))
print_dict(full_usages["coeditor.usage_analysis"], Path("full_usages.txt"))